# Extraccion de los datos 
 Se descomprime el json comprimido usando la funcion nativa de Pandas 2.0 read_json con los parametros. Luego se almacena en una variable de tipo dataframe con los datos brutos provenientes del json descomprimido

In [3]:
import numpy as np
import pandas as pd
import gzip
import os
import json
import ast

df_games = pd.read_json('/app/dataset/steam_games.json.gz', lines=True, compression='gzip')

print(df_games.head())
print(df_games.tail())



  publisher genres app_name title   url release_date  tags reviews_url specs  \
0      None   None     None  None  None         None  None        None  None   
1      None   None     None  None  None         None  None        None  None   
2      None   None     None  None  None         None  None        None  None   
3      None   None     None  None  None         None  None        None  None   
4      None   None     None  None  None         None  None        None  None   

  price  early_access  id developer  
0  None           NaN NaN      None  
1  None           NaN NaN      None  
2  None           NaN NaN      None  
3  None           NaN NaN      None  
4  None           NaN NaN      None  
              publisher                                 genres  \
120440  Ghost_RUS Games  [Casual, Indie, Simulation, Strategy]   
120441           Sacada              [Casual, Indie, Strategy]   
120442     Laush Studio            [Indie, Racing, Simulation]   
120443         SIXNAILS    

Hacemos un resumen consiso de los Dataframe

In [4]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [5]:
round(df_games.isna().sum().to_frame('% de Nulos')*100/len(df_games),2)

,% de Nulos
publisher,80.00
genres,76.05
app_name,73.32
title,75.02
url,73.32
release_date,75.04
tags,73.46
reviews_url,73.32
specs,73.88
price,74.46


In [6]:
# Describe todas las columnas del dataframe df_games
df_games.describe(include='all')


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135.000000,3.213300e+04,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,NaN,NaN,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,NaN,NaN,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,NaN,NaN,1259
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060588,4.517574e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.238577,1.827140e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000e+01,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.292800e+05,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,4.520600e+05,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,5.934000e+05,NaN


#### Descripción de los datos.
A partir del análisis exploratorio anterior, podemos observar que el conjunto de datos contiene 120445 registros y 13 variables con información sobre juegos de Steam. Sin embargo, hay una gran cantidad de datos nulos (75% en promedio). A continuación, se describen las variables:


- publisher: empresa publicadora del juego
- genres: género del juego. Esta formado por una lista de uno o mas géneros por registro
- app_name: nombre del juego
- title: título del juego
- url: URL de publicación del juego
- release_date: fecha de lanzamiento en formato 2018-01-04
- tags: etiquetas del juego. Esta formado por una lista de uno o más etiquetas por registro
- reviews_url: reviews del juego
- specs: especificaciones. Es una lista con uno o mas string con las especificaciones
- price: precio del juego
- early_access: indica el acceso temprano al juego con un True/False
- id: identificador único del juego
- developer: desarrollador del juego

# Transformación
En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas a partir de las existentes, la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

In [7]:
df_games.dropna(subset='id', inplace=True)
df_games.reset_index(drop=True, inplace=True)
df_games.shape

(32133, 13)

#### Verificación de duplicados
Del análisis exploratorio anterior notamos que el id 612880 esta duplicado. Se procede a verificar:

In [8]:
df_games[df_games['id'].duplicated(keep=False)]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13893,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
14572,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


In [9]:
# Eliminamos duplicados por id
df_games.drop_duplicates(subset='id', inplace=True)
df_games.shape

(32132, 13)

In [10]:
# Renombramos la columna 'id' a 'item_id'
df_games.rename(columns={'id': 'item_id'}, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer'],
      dtype='object')

#### Columnas title y app_name
##### Gestión de nulos
Notamos que ambas columnas contienen valores nulos que procedemos a reemplazar con 'unknown'.

In [11]:
columns = ['title', 'app_name']
df_games[columns] = df_games[columns].fillna('unknown')
df_games.isnull().sum()

publisher       8051
genres          3282
app_name           0
title              0
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
item_id            0
developer       3298
dtype: int64

In [12]:
# Realizamos la comparación y creamos una columna con el resultado.
df_games['is_equal'] = df_games['app_name'] == df_games['title']

# Calculamos el porcentaje de filas donde 'app_name' y 'title' son iguales
equal_percentage = round((df_games['is_equal'].sum() / len(df_games)) * 100, 2)

print(f"El porcentaje de filas donde 'app_name' y 'title' son iguales es: {equal_percentage}%")

El porcentaje de filas donde 'app_name' y 'title' son iguales es: 91.9%


In [13]:
# Filtramos las filas donde 'app_name' y 'title' no son iguales.
not_equal_df = df_games.loc[df_games['is_equal'] == False]
not_equal_df[['app_name', 'title']]

,app_name,title
4,Log Challenge,unknown
11,Icarus Six Sixty Six,unknown
19,After Life VR,unknown
20,Kitty Hawk,unknown
22,Mortars VR,unknown
...,...,...
32071,Tank of War-VR,unknown
32074,Flappy Arms,unknown
32075,SpaceWalker,unknown
32083,LIV Client,unknown


Cambiamos el nombre de la columna <code>app_name</code> por uno mas representativo como <code>item_name</code>.

In [14]:
df_games.rename(columns={'app_name': 'item_name'}, inplace=True)
df_games.columns

Index(['publisher', 'genres', 'item_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'item_id',
       'developer', 'is_equal'],
      dtype='object')

##### Tratamiento Columas <code>publisher</code>
Gestión de nulos
Notamos que también tenemos valores nulos para la columnas publisher y developer que reemplazamos con 'unknown'.

In [15]:
# Reemplazamos los valores nulos con 'unknown'
columns = ['publisher', 'developer']
df_games[columns] = df_games[columns].fillna('unknown')
df_games.isnull().sum()

publisher          0
genres          3282
item_name          0
title              0
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
item_id            0
developer          0
is_equal           0
dtype: int64

##### Tratamiento de <code>release_date</code>
Valor de nulos

In [16]:
import re
mask = df_games['release_date'].apply(lambda x: not re.match(r'\d{4}-\d{2}-\d{2}', str(x)))

- Filtramos las filas donde la máscara es True.

In [17]:
invalid_dates = df_games.loc[mask, 'release_date']
invalid_dates

4                None
10             Soon..
11               None
19               None
20               None
             ...     
32085     Coming Soon
32086            None
32095            2016
32121    January 2018
32132            None
Name: release_date, Length: 2351, dtype: object

<h1>Extracción del año.</h1>
Para los endpoints que se consumirán en la API solo necesitamos el año de lanzamiento del item, para ello lo extraemos de release_date de la siguiente manera:

- Creamos una máscara booleana donde release_date no coincide con el formato YYYY-MM-DD.

In [18]:
mask = df_games['release_date'].apply(lambda x: not re.match(r'\d{4}-\d{2}-\d{2}', str(x)))

# Filtramos las filas donde la máscara es True
invalid_dates = df_games.loc[mask, 'release_date']
invalid_dates

4                None
10             Soon..
11               None
19               None
20               None
             ...     
32085     Coming Soon
32086            None
32095            2016
32121    January 2018
32132            None
Name: release_date, Length: 2351, dtype: object

In [19]:
# Extraemos el año en una nueva columna y si no existe reemplazamos por 'unknown'.
df_games['release_year'] = df_games['release_date'].str.extract(r'(\d{4})').fillna('unknown')
df_games['release_year']

0           2018
1           2018
2           2017
3           2017
4        unknown
          ...   
32128       2018
32129       2018
32130       2018
32131       2017
32132    unknown
Name: release_year, Length: 32132, dtype: object

In [20]:
# Comprobamos que se haya extraido correctamente el año:
df_games[['release_date', 'release_year']].loc[[32121, 32095, 32085 ]]

,release_date,release_year
32121,January 2018,2018
32095,2016,2016
32085,Coming Soon,unknown


##### Buscamos años inexistentes en <code>release_year</code> y los pasamos a 'unknown'.

In [21]:
df_games['release_year'].unique()

array(['2018', '2017', 'unknown', '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '5275', '2019', '1975', '1970', '1980'], dtype=object)

In [22]:
df_games[df_games['release_year'] == "5275"]

,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,is_equal,release_year
13427,一次元创作组,"[Casual, Indie, Early Access]",Puzzle Sisters Foer,Puzzle Sisters Foer,http://store.steampowered.com/app/710190/Puzzl...,预热群52756441,"[Early Access, Casual, Indie]",http://steamcommunity.com/app/710190/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",None,1.0,710190.0,一次元创作组,True,5275


In [23]:
df_games['release_year'] = df_games['release_year'].replace('5275', 'unknown')
df_games['release_year'].unique()

array(['2018', '2017', 'unknown', '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '2019', '1975', '1970', '1980'], dtype=object)

#### Columnas de Precio
- Gestión de nulos

In [24]:
df_games[df_games['price'].isnull()][['price','genres', 'tags']].sample(10)

,price,genres,tags
26461,None,"[Indie, RPG, Strategy]","[Strategy, RPG, Indie]"
20119,None,"[Action, Adventure, Casual, Indie, Simulation,...","[Adventure, Casual, Indie, Simulation, Strateg..."
28660,None,"[Action, Casual, Indie, RPG]","[RPG, Indie, Casual, Action]"
3268,None,"[Action, Indie, Strategy]","[Strategy, Action, Indie]"
26213,None,"[Action, Adventure, Indie, Simulation]","[Adventure, Indie, Simulation, Action]"
29714,None,[Strategy],[Strategy]
4128,None,"[Indie, RPG, Sports, Strategy, Early Access]","[Early Access, Sports, Strategy, RPG, Indie]"
31245,None,"[Casual, Indie, Simulation]","[Simulation, Sandbox, Space, Physics, Science,..."
24950,None,"[Adventure, RPG]","[Adventure, RPG]"
23322,None,"[Casual, Indie]","[Indie, Casual]"


In [25]:
df_games.price.isnull().sum()

1377

In [26]:
# Crea una función para comprobar si un juego es gratis
def check_free(val):
    if isinstance(val, list):
        for item in val:
            if 'free' in item.lower():
                return True
    return False


# Aplicamos la función a cada fila del DataFrame
df_games['is_free'] = df_games['genres'].apply(check_free) | df_games['tags'].apply(check_free) | df_games['specs'].apply(check_free)

# Si un juego es gratuito y su precio es 'NaN', establece su precio en 0
df_games.loc[(df_games['is_free'] == True) & (df_games['price'].isnull()), 'price'] = 0

# Eliminamos la columna temporal 'is_free'
df_games.drop('is_free', axis=1, inplace=True)

Observamos que la cantidad de valores nulos ha disminuido, por lo que podemos concluir que teníamos varios juegos gratuitos con precios nulos, pero aún nos falta asignarle precio 0 a los juegos cuyo precio indica que es 'free' en la columna price.

In [27]:
df_games.price.isnull().sum()

1172

In [28]:
def replace_free_with_zero(val):
    if isinstance(val, str) and 'free' in val.lower():
        return 0
    return val

# Solo aplica la función a la columna 'price'
df_games['price'] = df_games['price'].apply(replace_free_with_zero)

In [29]:
df_games['price'].unique()

array([4.99, 0, 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, None, 10.99,
       1.5899999999999999, 14.99, 1.99, 59.99, 8.99, 6.99, 7.99, 39.99,
       19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99,
       3.49, 11.99, 13.99, 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99, 54.99,
       74.99, 'Install Theme', 0.89, 'Third-party', 0.5, 'Play Now',
       299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99, 10.93,
       1.3900000000000001, 36.99, 4.49, 2.0, 4.0, 9.0, 234.99,
       1.9500000000000002, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3,
       0.9500000000000001, 172.24, 249.99, 2

In [30]:
non_price_values = ['Install Now', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Play Now', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00']
df_games[df_games['price'].isin(non_price_values)]

,publisher,genres,item_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,is_equal,release_year
2404,EVGA,[Utilities],EVGA Precision XOC,EVGA Precision XOC,http://store.steampowered.com/app/268850/EVGA_...,2014-09-19,"[Utilities, Software, Free to Play]",http://steamcommunity.com/app/268850/reviews/?...,"[Single-player, Steam Achievements]",Install Now,0.0,268850.0,EVGA,True,2014
2870,Privateer Press Interactive,"[Indie, Strategy]",WARMACHINE: Tactics,WARMACHINE: Tactics,http://store.steampowered.com/app/253510/WARMA...,2014-11-20,"[Strategy, Turn-Based, Turn-Based Strategy, St...",http://steamcommunity.com/app/253510/reviews/?...,"[Single-player, Multi-player, Cross-Platform M...",Play WARMACHINE: Tactics Demo,0.0,253510.0,WhiteMoon Dreams,True,2014
3831,unknown,"[Adventure, Casual, Indie, RPG, Simulation]",FREE China Theme Pack,FREE China Theme Pack,http://store.steampowered.com/app/370880/FREE_...,2015-06-10,"[Adventure, RPG, Indie, Casual, Simulation]",http://steamcommunity.com/app/370880/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",Install Theme,0.0,370880.0,Stolen Couch Games,True,2015
3917,unknown,[Indie],Parcel - Soundtrack,Parcel - Soundtrack,http://store.steampowered.com/app/362970/Parce...,2015-07-02,[Indie],http://steamcommunity.com/app/362970/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",Third-party,0.0,362970.0,Polar Bunny Ltd,True,2015
4025,DigitalEZ,"[Casual, Indie]",Oblivious Garden ~White Day,Oblivious Garden ~White Day,http://store.steampowered.com/app/345040/Obliv...,2015-07-20,"[Casual, Indie]",http://steamcommunity.com/app/345040/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",Play Now,0.0,345040.0,"CorypheeSoft,DigitalEZ",True,2015
22733,Boomzap Entertainment,[Strategy],Legends of Callasia,Legends of Callasia,http://store.steampowered.com/app/438920/Legen...,2016-06-10,"[Strategy, Wargame, Fantasy, Multiplayer, Turn...",http://steamcommunity.com/app/438920/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Play the Demo,0.0,438920.0,Boomzap Entertainment,True,2016
24999,unknown,None,Syber Steam Machine,Syber Steam Machine,http://store.steampowered.com/app/353420/Syber...,2015-11-10,"[Steam Machine, Hardware]",http://steamcommunity.com/app/353420/reviews/?...,None,Starting at $499.00,0.0,353420.0,unknown,True,2015
25000,unknown,None,Alienware Steam Machine,Alienware Steam Machine,http://store.steampowered.com/app/353390/Alien...,2015-11-10,"[Steam Machine, Hardware, Gaming, Futuristic, ...",http://steamcommunity.com/app/353390/reviews/?...,None,Starting at $449.00,0.0,353390.0,unknown,True,2015
26216,unknown,"[Adventure, Casual, Indie, Simulation]",Area-X - Extra Gallery,Area-X - Extra Gallery,http://store.steampowered.com/app/383860/AreaX...,2015-06-24,"[Adventure, Indie, Casual, Simulation]",http://steamcommunity.com/app/383860/reviews/?...,"[Single-player, Downloadable Content]",Play Now,0.0,383860.0,Zeiva Inc,True,2015
31836,"PopCap Games, Inc.",[Casual],Peggle Extreme,Peggle Extreme,http://store.steampowered.com/app/3483/Peggle_...,2007-09-11,"[Casual, Puzzle, Free to Play, Action]",http://steamcommunity.com/app/3483/reviews/?br...,[Single-player],Third-party,0.0,3483.0,"PopCap Games, Inc.",True,2007


Transformamos los valores que no representan un precio en 0.

In [31]:
df_games.loc[df_games['price'] == 'Starting at $499.00', 'price'] = 499
df_games.loc[df_games['price'] == 'Starting at $449.00', 'price'] = 449
non_price_values = ['Install Now', 'Play WARMACHINE: Tactics Demo', 'Install Theme', 'Third-party', 'Play Now', 'Play the Demo']
df_games.loc[df_games['price'].isin(non_price_values), 'price'] = 0

Se calcula la media, la moda y la mediana de la columna price para imputar los nulos restantes.

In [32]:
media = df_games['price'].mean()
moda = df_games['price'].mode()[0]
mediana = df_games['price'].median()

print(f"La media de los precios de juegos Steam es {media:.2f}")
print(f"La moda de los precios de juegos Steam es {moda:.2f}")
print(f"La mediana de los precios de juegos Steam es {mediana:.2f}")

La media de los precios de juegos Steam es 8.84
La moda de los precios de juegos Steam es 4.99
La mediana de los precios de juegos Steam es 4.99


Inputamos los nulos con la mediana.

In [33]:
df_games['price'].fillna(df_games['price'].median(), inplace=True)
df_games.isnull().sum()

/tmp/ipykernel_446/4128492194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games['price'].fillna(df_games['price'].median(), inplace=True)
/tmp/ipykernel_446/4128492194.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_games['price'].fillna(df_games['price'].median(), inplace=True)


publisher          0
genres          3282
item_name          0
title              0
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price              0
early_access       0
item_id            0
developer          0
is_equal           0
release_year       0
dtype: int64

#### Columnas de Generos, Tags y Especificaciones
- Gestión de nulos

In [34]:
df_games[df_games['genres'].isnull()][['genres','tags', 'specs']].sample(10)

,genres,tags,specs
21327,None,"[Indie, VR, Rhythm, Music]","[Single-player, HTC Vive, Oculus Rift, Tracked..."
7611,None,"[Action, Adventure, Simulation, VR]","[Single-player, Full controller support, HTC V..."
23400,None,"[Movie, Satire, Comedy, Music, Documentary]",[Captions available]
8452,None,"[Indie, Casual, VR]","[Single-player, Multi-player, Local Multi-Play..."
18118,None,"[Casual, Simulation, Strategy, Adventure, Indi...","[Single-player, Full controller support, HTC V..."
4745,None,"[Tutorial, Design & Illustration]",None
19691,None,"[Adventure, Indie, Casual, Action, Strategy, V...","[Single-player, Steam Trading Cards, HTC Vive,..."
8947,None,"[Early Access, Design & Illustration]","[Includes level editor, HTC Vive, Tracked Moti..."
4017,None,"[Drama, Movie, Comedy, Indie]",None
8290,None,"[Early Access, Action, Simulation, Indie, Casu...","[Single-player, Multi-player, Online Multi-Pla..."


In [35]:
df_games.genres.isnull().sum()

3282

In [36]:
# Exploramos el contenido de genres, tags y specs.

columns = ['genres', 'tags','specs']
df_games[columns].sample(10)

,genres,tags,specs
130,"[Casual, Simulation]","[Casual, Simulation]",[Single-player]
223,"[Action, Indie]","[Indie, Platformer, Cute, Action, Puzzle, Casu...","[Single-player, Steam Achievements]"
23405,None,"[Movie, Drama, War]",[Captions available]
2103,"[Adventure, Casual]","[Adventure, Casual, Point & Click, Mystery, Fi...",[Single-player]
23541,"[Action, Indie, Sports]","[Great Soundtrack, Action, Local Multiplayer, ...","[Single-player, Local Multi-Player, Local Co-o..."
3005,[Simulation],[Simulation],"[Single-player, Multi-player, Cross-Platform M..."
16679,"[Action, Indie]","[Sexual Content, Nudity, Violent, Action, Indie]","[Single-player, Multi-player, Shared/Split Scr..."
3967,"[Action, Adventure, Indie]","[Action, Adventure, Indie]",[Downloadable Content]
23967,"[Adventure, Indie, RPG]","[Adventure, RPG, Indie]","[Single-player, Multi-player, Co-op, Shared/Sp..."
9591,"[Casual, Strategy]","[Casual, Strategy]","[Single-player, In-App Purchases]"


Podemos observar que tags contiene tambien a genres por lo que podemos usarla para imputar los nulos en genres.

In [37]:
# Creamos una lista de géneros únicos
unique_genres = df_games['genres'].explode().unique()

# Definimos una función para buscar géneros en tags
def find_genres(row):
    if isinstance(row['tags'], list):
        genres_in_tags = [tag for tag in row['tags'] if tag in unique_genres]
        if genres_in_tags:
            return genres_in_tags
    return row['genres']

# Aplicamos la función a cada fila y almacenamos en la columna temporal 'genres_2'
df_games['genres_2'] = df_games.apply(find_genres, axis=1)
df_games[df_games['genres'].isnull()][['genres', 'genres_2', 'tags']].head()

,genres,genres_2,tags
4,None,"[Action, Indie, Casual, Sports]","[Action, Indie, Casual, Sports]"
11,None,[Casual],[Casual]
19,None,"[Early Access, Indie]","[Early Access, Indie, VR]"
20,None,"[Early Access, Action, Adventure, Indie, Casual]","[Early Access, Action, Adventure, Indie, Casual]"
22,None,"[Early Access, Strategy, Action, Indie, Casual]","[Early Access, Strategy, Action, Indie, Casual..."


Eliminamos la columna 'genres' para posteriormente reemplazarla por la columna 'genres_2'. y tambien se eliminará la columna 'tags' y la 'specs' porque no aportan información.

In [38]:

columns = ['genres', 'tags', 'specs']
df_games.drop(columns, axis=1, inplace=True)
# Renombramos la columna genres_2 como genres
df_games = df_games.rename(columns={'genres_2': 'genres'})
df_games.columns

Index(['publisher', 'item_name', 'title', 'url', 'release_date', 'reviews_url',
       'price', 'early_access', 'item_id', 'developer', 'is_equal',
       'release_year', 'genres'],
      dtype='object')

#### Columnas de URL y Reviews URL


In [39]:
import pyarrow

# Elimina las columnas 'url' y 'reviews_url' del dataframe.
columns = ['url', 'reviews_url']
df_games.drop(columns, axis=1, inplace=True)

# Cambia el tipo de dato de la columna 'early_access' a booleano.
df_games['early_access'] = df_games['early_access'].astype(bool)
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32132
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     32132 non-null  object 
 1   item_name     32132 non-null  object 
 2   title         32132 non-null  object 
 3   release_date  30066 non-null  object 
 4   price         32132 non-null  float64
 5   early_access  32132 non-null  bool   
 6   item_id       32132 non-null  float64
 7   developer     32132 non-null  object 
 8   is_equal      32132 non-null  bool   
 9   release_year  32132 non-null  object 
 10  genres        31102 non-null  object 
dtypes: bool(2), float64(2), object(7)
memory usage: 3.5+ MB


# Carga

##### Cargamos el dataset con el dataset limpio y formato comprimido.

In [40]:
# Definimos la ruta de destino y el nombre del archivo.
folder_path = '../dataset/'
file_name = 'steam_games.parquet'

# Verificamos si el folder_path existe
if not os.path.exists(folder_path):
    # Si no existe, lo creamos
    os.makedirs(folder_path)

# Exportamos el DataFrame a un archivo Parquet
df_games.to_parquet(os.path.join(folder_path, file_name), engine='pyarrow', compression='zstd')

print(f'El archivo {file_name} se guardó correctamente en {folder_path}')

El archivo steam_games.parquet se guardó correctamente en ../dataset/
